# Philip Carr
# CS/CNS/EE_156a_Homework_2_Code_Part_2 (Jupyter Notebook)

Code for The Linear Regression Algorithm (Problems 5 - 10)

In [1]:
import random as rn
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def sign(x):
    '''
    Return the sign of a number (1 if positive, 0 if 0, or -1 if
    negative).
    
    Return type: int
    '''
    if x >= 0:
        return 1
    elif x == 0:
        return 0
    else:
        return -1

In [3]:
def target_function_line_value(x_point, y_point, m, b,
                               sign_orientation):
    '''
    Return the evaluation of the target function (line with slope m
    and y-intercept b) for a given point (1, x_point, y_point) and
    sign_orientation: determines whether points have value of 1 or -1
    when above the target function line with this value being either
    1 or -1 respectively.
    
    Return type: int
    '''
    y_line = m * x_point + b
    if y_point >= y_line:
        return 1 * sign_orientation
    else:
        return -1 * sign_orientation

In [4]:
def get_target_function_line():
    '''
    Return a randomly generated target function line's slope (m) and
    y-intercept (b).
    
    Return type: tuple of floats
    '''
    x1 = rn.uniform(-1, 1)
    y1 = rn.uniform(-1, 1)
    x2 = rn.uniform(-1, 1)
    y2 = rn.uniform(-1, 1)
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    return m, b

In [5]:
def get_target_function():
    '''
    Return a randomly generated target function line.
    
    Return type: function that takes two floats and returns an int
    (instance of the target_function_line_value function).
    '''
    sign_orientation = rn.choice([-1, 1])
    m, b = get_target_function_line()
    return m, b, lambda x_point, y_point: \
        target_function_line_value(x_point, y_point, m, b,
                                   sign_orientation)

In [6]:
def get_random_points(N, non_linear_transform=False):
    '''
    Return a list of randomly generated points within
    the region [-1, 1] x [-1, 1].
    
    Return type: list of points (each point is a list)
    '''
    random_points = []
    for n in range(N):
        x0 = 1.0 # artificial coordinate
        x = rn.uniform(-1, 1)
        y = rn.uniform(-1, 1)
        if not non_linear_transform:
            random_points.append([x0, x, y])
        else:
            xy = x * y
            x_squared = x * x
            y_squared = y * y
            random_points.append([x0, x, y, xy, x_squared,
                                  y_squared])
    return random_points

In [7]:
def get_point_values(points, mode='target_function',
                     target_function=None,
                     target_function_noise_rate=0, la=None):
    '''
    Return the values corresponding to the (list of) points given.
    If the mode is 'target_function', then the returned values are
    function evaluations of the target function target_function of
    the given points. If the mode is 'la', then the returned values
    are the given learning algorithm hypothesis function evaluations
    of the given points.
    
    Default arguments:
    mode: either 'target_function' or 'la'.
    la: None, or an instance of a learning algorithm class if mode
    is 'la'. target_function: None, or an instance of a target
    function if mode is 'target_function'.
    
    Return type: list of ints
    '''
    point_values = []
    for i in range(len(points)):
        if mode == 'target_function':
            value = target_function(points[i][1], points[i][2])
            flip_sign = rn.random()
            if flip_sign < target_function_noise_rate:
                value *= -1
            point_values.append(value)
        else:
            point_values.append(la.evaluate(points[i]))
    return point_values

In [8]:
def plot_graph(points, values, mode='target_function', la=None,
               m_b=None):
    '''
    Plot the given (list of) points and (a list of) their
    corresponding values. If the mode is 'target_function', then the
    values should be function evaluations of the target function
    target_function of the given points, and the target function
    line is also plotted. If the mode is 'la', then the values
    should be the learning algorithm hypothesis function evaluations
    of the given points, and additional fainter points are plotted to
    display the learning algorithm's hypothesis function.
    
    Default arguments:
    mode: either 'target_function' or 'pla'.
    la: None, or an instance of a learning algorithm class if mode is
        'la'.
    m_b: None, or a tuple of the slope and y-intercept of the target
        function line if mode is 'target_function'.
    
    Return type: None
    '''
    marker_modes = {'target_function':'o', 'la':'x'}
    title_extra = "(line in black)" * (m_b != None)
    mode_titles = {'target_function':('Random points and target '
                                      + 'function ' + title_extra),
                   'la':'learning algorithm points (original points '
                        'are x\'s)'}
    plt.figure()
    plt.clf()
    plt.ylim(-1, 1)
    plt.title(mode_titles[mode])
    for i in range(len(points)):
        if values[i] == -1:
            plt.scatter(points[i][1], points[i][2],
                        color='red', marker=marker_modes[mode])
        elif values[i] == 0:
            plt.scatter(points[i][1], points[i][2],
                        color='yellow', marker=marker_modes[mode])
        else:
            plt.scatter(points[i][1], points[i][2],
                        color='blue', marker=marker_modes[mode])
    
    if mode == 'target_function':
        line_x_values = np.linspace(-1, 1, 1000)
        if m_b != None:
            plt.plot(line_x_values, m_b[0] * line_x_values + m_b[1],
                     color='black')
    else:
        more_random_points = get_random_points(1000)
        for i in range(len(more_random_points)):
            if sign(la.evaluate(more_random_points[i])) == -1:
                plt.scatter(more_random_points[i][1],
                            more_random_points[i][2],
                            color='red', alpha=0.2)
            elif sign(la.evaluate(more_random_points[i])) == 0:
                plt.scatter(more_random_points[i][1],
                            more_random_points[i][2],
                            color='yellow', alpha=0.2)
            else:
                plt.scatter(more_random_points[i][1],
                            more_random_points[i][2],
                            color='blue', alpha=0.2)

In [9]:
class PLA:
    '''
    This class represents the PLA (Perceptron Learning Algorithm).
    This class contains the weights, as well as methods for running
    the PLA.
    '''
    def __init__(self, initial_weights=[0] * 3):
        '''
        Initialize the weights of the PLA using the given initial_weights.
        
        Return type: class (PLA)
        '''
        self.weights = initial_weights
    
    def __repr__(self):
        '''
        Print the weights of the PLA.
        
        Return type: string
        '''
        print('PLA weights: %s' % self.weights)
    
    def get_weights(self):
        '''
        Return the weights of the PLA.
        
        Return type: list of floats
        '''
        return self.weights
    
    def evaluate(self, point):
        '''
        Return the PLA hypothesis function evaluation of the given
        point.
        
        Return type: int
        '''
        value = 0
        for i in range(len(self.weights)):
            value += self.weights[i] * point[i]
        return sign(value)
    
    def iterate(self, point, value):
        '''
        Update the PLA weights using the given point and its
        corresponding target function value.
        
        Return type: None
        '''
        for i in range(len(self.weights)):
            self.weights[i] += value * point[i]
    
    def get_missclassified_indices(self, points, values,
                                   debug=False):
        '''
        Return the list of indices of the points list corresponding
        to the (list of) points that the PLA missclassifies (when the
        target function and PLA hypothesis function evaluations of a
        point disagree (where values is the list of target function
        point values of the given points)).
        
        Default arguments:
        debug: If True, plot a graph of the points according to
            their classifications made by the PLA with the current
            weights. Otherwise, this plot is not made.
        
        Return type: None
        '''
        missclassified_indices = []
        for i in range(len(points)):
            if self.evaluate(points[i]) != values[i]:
                missclassified_indices.append(i)
        
        if debug:
            point_values = get_point_values(points, mode='la',
                                            la=self)
            plot_graph(points, point_values, mode='pla',
                       pla=self)
        
        return missclassified_indices
    
    def run(self, points, values, debug=False):
        '''
        Return the number of iterations it takes to converge the
        PLA to correctly classify all the given (list of) points and
        (the list of) target function point values of the given
        points (values).
        
        Default arguments:
        debug: If True, plot graphs of the points according to
            their classifications made by the PLA with every
            iteration of the weights. Otherwise, none of these plots
            are made.
        
        Return type: int
        '''
        iteration_count = 0
        missclassified_indices = \
            self.get_missclassified_indices(points, values,
                                            debug=debug)
        while missclassified_indices != [] and \
              iteration_count < 100:
            missclassified_index = rn.choice(missclassified_indices)
            self.iterate(points[missclassified_index],
                         values[missclassified_index])
            iteration_count += 1
            missclassified_indices = \
                self.get_missclassified_indices(points, values,
                                                debug=debug)
        return iteration_count

In [10]:
def run_pla(pla, N, print_statements=False, debug=False):
    '''
    Return the number of iterations for the given PLA algorithm to
    converge using a dataset of N randomly generated points with
    classifications determined by a randomly generated target
    function line.
    
    Default arguments:
    print_statements: If True, print information about the
        performance metrics of the PLA convergence.
    debug: if True, plot a graph of the randomly generated points
        colored by their classifications with the target function
        line and plot graphs of the points according to their
        classifications made by the PLA with every iteration of the
        weights. Otherwise, none of these plots are made.
    
    Return value: tuple of an int and a function
    '''
    if print_statements:
        print('Running Perceptron Learning Algorithm '
              + ('with N = %d points' % N))
    m, b, target_function = get_target_function()
    random_points = get_random_points(N)
    point_values = get_point_values(random_points,
                                    mode='target_function',
                                    target_function=target_function)
    if debug:
        plot_graph(random_points, point_values,
                   mode='target_function', m_b=(m, b))
    iteration_count = pla.run(random_points, point_values,
                              debug=debug)
    if print_statements:
        print('Perceptron Learning Algorithm convergence time: '
              + ('%d iterations' % iteration_count))
    return iteration_count, target_function

In [11]:
def test_pla(pla, N, trials=1000, n_prob_points=10000, debug=False):
    '''
    Print the average number of iterations for the given PLA
    algorithm to converge and the probability that f
    (target function) and g (hypothesis function) will disagree on
    their classification of a random point (P[f(x) != g(x)]) using
    datasets of N randomly generated points with classifications
    determined by randomly generated target function lines.
    
    Default arguments:
    trials: Number of trials of which to take the average PLA
        performance metrics.
    n_prob_points: Number of additional points used to approximate
        P[f(x) != g(x)].
    debug: if True, plot a graph of the randomly generated points
        colored by their classifications with the target function
        line and plot graphs of the points according to their
        classifications made by the PLA with every iteration of the
        weights. Otherwise, none of these plots are made.
    
    Return value: None
    '''
    total_iterations = 0
    total_f_neq_g = 0
    for i in range(trials):
        iteration_count, target_function = run_pla(pla, N)
        total_iterations += iteration_count
        random_points = get_random_points(n_prob_points)
        point_values = \
            get_point_values(random_points, mode='target_function',
                             target_function=target_function)
        total_f_neq_g += \
            len(pla.get_missclassified_indices(random_points,
                                               point_values))
    print(('Average convergence time for N = %d ' % N)
          + ('points over %d trials: %f iterations.'
             % (trials, total_iterations / trials)))
    print(('Average P[f(x) != g(x)] for N = %d ' % N)
          + ('points over %d trials: %f.'
             % (trials, total_f_neq_g / n_prob_points / trials)))

In [12]:
#pla_10 = PLA()
#pla_100 = PLA()

#test_pla(pla_10, 10)
#test_pla(pla_100, 100)

In [13]:
class LRA:
    '''
    This class represents the LRA (Linear Regression Algorithm).
    This class contains the weights, as well as methods for running
    the LRA.
    '''
    def __init__(self, initial_weights=np.array([0] * 3)):
        '''
        Initialize the weights of the LRA, given initial_weights.
        
        Return type: class (LRA)
        '''
        self.weights = initial_weights
    
    def __repr__(self):
        '''
        Print the weights of the LRA.
        
        Return type: string
        '''
        print('PLA weights: %s' % self.weights)
    
    def get_weights(self):
        '''
        Return the weights of the LRA.
        
        Return type: list of floats
        '''
        return self.weights
    
    def evaluate(self, point):
        '''
        Return the LRA hypothesis function evaluation of the given
        point.
        
        Return type: int
        '''
        value = 0
        for i in range(len(self.weights)):
            value += self.weights[i] * point[i]
        return value
    
    def error(self, points, values):
        '''
        Return the error of a given linear regression algorithm given a 
        list of points and their corresponding (true) values.
        
        Return type: float
        '''
        N = len(points)
        h_x_array = np.array(get_point_values(points, mode='la',
                                              la=self))
        y_array = np.array(values)
        return (1 / N) * np.sum((h_x_array - y_array)
                                * (h_x_array - y_array))
    
    def estimate_weights(self, points, values):
        '''
        Use the linear regression algorithm to estimate the weights
        of the LRA by computing w = X^T y, where X^T is the pseudo-
        inverse of X and where X is the matrix of points in the
        dataset, given a list of points and their corresponding
        values. 
        
        Return type: None
        '''
        X = np.array(points)
        y = np.array(values)
        X_transpose = np.transpose(X)
        X_T_X = np.matmul(X_transpose, X)
        X_T_X_inverse = np.linalg.inv(X_T_X)
        X_pseudo_inverse = np.matmul(X_T_X_inverse, X_transpose)
        self.weights = np.dot(X_pseudo_inverse, y)

In [14]:
def run_lra(lra, N, linear=True, target_function_noise_rate=0,
            non_linear_transform=False, debug=False):
    '''
    Return the in-sample error of an LRA training on randomly
    generated data determined by a randomly generated target
    function, the randomly generated data, and the randomly
    generated target function.
    
    Default argument:
    debug: if True, plot a graph of the randomly generated points
        colored by their classifications with the target function
        line, as well as the LRA classifications of the data (if
        no nonlinear transformation is done to the data).
    
    Return type: tuple of float and function
    '''
    if linear:
        m, b, target_function = get_target_function()
    else:
        target_function = lambda x1, x2: \
            sign(x1 * x1 + x2 * x2 - 0.6)
    
    random_points = get_random_points(
        N, non_linear_transform=non_linear_transform)
    point_values = get_point_values(
        random_points, mode='target_function',
        target_function=target_function,
        target_function_noise_rate=target_function_noise_rate)
    
    if debug:
        if linear:
            plot_graph(random_points, point_values,
                       mode='target_function', m_b=(m, b))
        else:
            plot_graph(random_points, point_values,
                       mode='target_function')
    lra.estimate_weights(random_points, point_values)
    E_in = lra.error(random_points, point_values)
    if debug and (not non_linear_transform):
        plot_graph(random_points, point_values, mode='la', la=lra)
    return E_in, random_points, target_function

In [15]:
def test_lra(lra, N, trials=1000, n_out_sample_points=1000,
             test_pla=False, linear=True,
             target_function_noise_rate=0,
             non_linear_transform=False, debug=False):
    '''
    Print the in-sample and out-of-sample errors produced
    using an LRA, as well as the weights of the LRA if
    a nonlinear transformation is used for the dataset.
    PLA convergence time using trained LRA weights is also
    displayed if test_pla is True.
    
    Default arguments:
    trials: Number of trials of which to take the average LRA
        performance metrics.
    debug: if True, plot a graph of the randomly generated points
        colored by their classifications with the target function
        line, as well as the LRA classifications of the data.
    
    Return type: None
    '''
    total_E_in = 0
    total_E_out = 0
    total_f_neq_g = 0
    if not non_linear_transform:
        weight_totals = np.array([0.] * 3)
    else:
        weight_totals = np.array([0.] * 6)
    for i in range(trials):
        E_in, random_points, target_function = run_lra(
            lra, N, linear=linear,
            target_function_noise_rate=target_function_noise_rate,
            non_linear_transform=non_linear_transform,
            debug=debug)
        weight_totals += lra.get_weights()
        total_E_in += E_in
        random_points_out_sample = get_random_points(
            n_out_sample_points, non_linear_transform)
        point_values = get_point_values(
            random_points_out_sample, mode='target_function',
            target_function=target_function,
            target_function_noise_rate=target_function_noise_rate)
        total_E_out += lra.error(random_points_out_sample,
                                 point_values)
    
    if non_linear_transform:
        print(("Average weights for N = %d " % N)
              + ('points over %d trials: %s.'
                 % (trials, weight_totals / trials)))
    print(('Average E_in (in-sample error) for N = %d ' % N)
          + ('points over %d trials: %f.'
             % (trials, total_E_in / trials)))
    print(('Average E_out (out-of-sample error) for N = %d '
           % n_out_sample_points)
          + ('points trained with %d points over %d trials: %f.'
             % (N, trials, total_E_out / trials)))
    
    if test_pla:
        total_iterations = 0
        for i in range(trials):
            E_in, random_points, target_function = run_lra(lra, N)
            pla = PLA(initial_weights=lra.get_weights())
            point_values = get_point_values(
                random_points, mode='target_function',
                target_function=target_function,
                target_function_noise_rate=
                    target_function_noise_rate)
            total_iterations += pla.run(random_points, point_values)
        print(('Average convergence time for N = %d ' % N)
              + ('points over %d trials: %f iterations.'
                 % (trials, total_iterations / trials)))

For Problems 5 and 6

In [16]:
lra_100 = LRA()
test_lra(lra_100, 100)

Average E_in (in-sample error) for N = 100 points over 1000 trials: 0.285025.
Average E_out (out-of-sample error) for N = 1000 points trained with 100 points over 1000 trials: 0.303925.


For Problem 7

In [17]:
lra_10 = LRA()
test_lra(lra_10, 10, test_pla=True)

Average E_in (in-sample error) for N = 10 points over 1000 trials: 0.210663.
Average E_out (out-of-sample error) for N = 1000 points trained with 10 points over 1000 trials: 0.452106.
Average convergence time for N = 10 points over 1000 trials: 3.672000 iterations.


For Problem 8

In [18]:
lra_1000 = LRA()
test_lra(lra_1000, 1000, linear=False,
         target_function_noise_rate=0.1)

Average E_in (in-sample error) for N = 1000 points over 1000 trials: 0.995000.
Average E_out (out-of-sample error) for N = 1000 points trained with 1000 points over 1000 trials: 1.000912.


For Problems 9 and 10

In [19]:
lra_1000 = LRA()
test_lra(lra_1000, 1000, linear=False,
         target_function_noise_rate=0.1, non_linear_transform=True)

Average weights for N = 1000 points over 1000 trials: [-9.93329515e-01 -2.95465596e-03  6.44966089e-04  3.55773173e-03
  1.56048685e+00  1.55803626e+00].
Average E_in (in-sample error) for N = 1000 points over 1000 trials: 0.564534.
Average E_out (out-of-sample error) for N = 1000 points trained with 1000 points over 1000 trials: 0.572992.
